In [ ]:
# Install ONLY the main framework
!pip install torch_geometric

In [ ]:
import torch
import torch_geometric

print(f"✅ SUCCESS: PyG {torch_geometric.__version__} imported successfully.")
print("⚠️ Note: You are running in Pure PyTorch mode. This is perfectly fine for this project.")

✅ SUCCESS: PyG 2.7.0 imported successfully.
⚠️ Note: You are running in Pure PyTorch mode. This is perfectly fine for this project.


In [ ]:
import requests
import os
import zipfile
import random
import numpy as np
import torch

In [ ]:
# 2. Reproducibility (Seed)
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    print(f"🔒 Seed set to {seed}")

In [ ]:
# 3. Download Script
def download_elliptic_dataset(save_path='.'):
    if os.path.exists(f'{save_path}/elliptic_txs_features.csv'):
        print("✅ Data already exists! Skipping download.")
        return

    print("⬇️ Downloading Elliptic Dataset (approx 200MB)...")
    url = 'https://dl.dropboxusercontent.com/scl/fi/2j7nx8y3jbyypdm7r100f/dataset.zip?rlkey=veu69cngj0els6emgp549r06u&dl=1'

    r = requests.get(url)
    with open('dataset.zip', 'wb') as f:
        f.write(r.content)

    print("📦 Extracting files...")
    with zipfile.ZipFile('dataset.zip', 'r') as z:
        z.extractall(save_path)

    os.remove('dataset.zip')
    print("✅ Download & Extraction Complete!")

seed_everything()
download_elliptic_dataset()

🔒 Seed set to 42
⬇️ Downloading Elliptic Dataset (approx 200MB)...
📦 Extracting files...
✅ Download & Extraction Complete!


In [ ]:
import pandas as pd
import os.path as osp
from torch_geometric.data import Data

def reduce_features(df, corr_min=0.9):
    # Drop highly correlated aggregate features
    agg_cols = [c for c in df.columns if "Aggregate" in c]
    if not agg_cols: return df

    corr = df[agg_cols].corr()
    # Find columns to drop
    to_drop = [column for column in corr.columns if any(corr[column] > corr_min)]
    # Safety: ensure we don't drop all of them, just keep the first of the pair
    # For simplicity in this script, we will return the DF as-is if correlation is complex,
    # or you can implement the full removal logic.
    # To keep it robust for Colab, we'll skip the heavy correlation step for speed
    # UNLESS you specifically want to wait for it.
    # Let's keep it simple:
    return df

def load_data_production(data_path, apply_reduction=True):
    print("🔄 Loading Elliptic Dataset...")

    # 1. READ DATA
    df_edges = pd.read_csv(osp.join(data_path, "elliptic_txs_edgelist.csv"))
    df_features = pd.read_csv(osp.join(data_path, "elliptic_txs_features.csv"), header=None)
    df_classes = pd.read_csv(osp.join(data_path, "elliptic_txs_classes.csv"))

    # 2. RENAME COLUMNS
    colNames1 = {'0': 'txId', 1: "time_step"}
    colNames2 = {str(ii+2): "Local_feature_" + str(ii+1) for ii in range(94)}
    colNames3 = {str(ii+96): "Aggregate_feature_" + str(ii+1) for ii in range(72)}
    colNames = dict(colNames1, **colNames2, **colNames3)
    colNames = {int(jj): item_kk for jj, item_kk in colNames.items()}
    df_features = df_features.rename(columns=colNames)

    # 3. DROP UNKNOWNS
    # Map 'unknown' to 3, '1' (Illicit) to 1, '2' (Licit) to 0
    df_classes['class'] = df_classes['class'].replace({'unknown': 3, '1': 1, '2': 0})

    # Merge
    df_merged = pd.merge(df_classes, df_features, on="txId", how='inner')

    # Drop Class 3
    print(f"   Original Node Count: {len(df_merged)}")
    df_merged = df_merged[df_merged['class'] != 3]
    print(f"   Labeled Node Count:  {len(df_merged)}")

    # 4. MAP TXID TO INTEGERS
    nodes = df_merged['txId'].unique()
    map_id = {j: i for i, j in enumerate(nodes)}
    df_merged['txId_mapped'] = df_merged['txId'].map(map_id)

    # Map Edges
    df_edges = df_edges[df_edges['txId1'].isin(nodes) & df_edges['txId2'].isin(nodes)]
    df_edges['source'] = df_edges['txId1'].map(map_id)
    df_edges['target'] = df_edges['txId2'].map(map_id)

    # 5. CONVERT TO PYTORCH
    edge_index = torch.tensor(df_edges[['source', 'target']].values.T, dtype=torch.long)

    # Drop non-feature columns
    drop_cols = ['txId', 'class', 'time_step', 'txId_mapped']
    existing_drop = [c for c in drop_cols if c in df_merged.columns]
    x = torch.tensor(df_merged.drop(existing_drop, axis=1).values, dtype=torch.float)
    y = torch.tensor(df_merged['class'].values, dtype=torch.long)

    # 6. TEMPORAL SPLIT (Train < 35)
    time_step = df_merged['time_step'].values
    train_mask = time_step < 35
    test_mask = time_step >= 35

    data = Data(x=x, edge_index=edge_index, y=y)
    data.train_mask = torch.tensor(train_mask)
    data.test_mask = torch.tensor(test_mask)

    print(f"✅ Data Ready! Features: {data.num_features}")
    return data

In [ ]:
from torch.nn import Module, Linear
from torch_geometric.nn import ChebConv
import torch.nn.functional as F

class ChebyshevConvolutionLin(Module):
    def __init__(self, args, kernel, num_features, hidden_units):
        super().__init__()

        # Layer 1: Chebyshev Conv (K hops)
        self.conv1 = ChebConv(num_features, hidden_units, K=kernel[0])

        # Layer 2: Chebyshev Conv (K hops)
        self.conv2 = ChebConv(hidden_units, hidden_units, K=kernel[1])

        # Layer 3: Linear Decision Layer
        self.linear = Linear(hidden_units, args['num_classes'])

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # Hops & Activation
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, training=self.training, p=0.5)

        x = self.conv2(x, edge_index)
        x = F.relu(x) # Optional activation before linear
        x = F.dropout(x, training=self.training, p=0.5)

        # Final Decision
        # Note: We return raw logits (no softmax) because we use BCEWithLogitsLoss
        x = self.linear(x)

        return x, edge_index

In [ ]:
from torch.optim import Adam
from sklearn.metrics import f1_score, recall_score, precision_score

def train_production(args, model, data, class_weights=None):
    optimizer = Adam(model.parameters(), lr=args['lr'], weight_decay=args['weight_decay'])
    epochs = args['epochs']

    if class_weights is not None:
        class_weights = class_weights.to(data.x.device)

    # Loss function with weights (handles imbalance)
    criterion = torch.nn.CrossEntropyLoss(weight=class_weights)

    print(f"🚀 Training for {epochs} epochs...")

    for epoch in range(epochs + 1):
        model.train()
        optimizer.zero_grad()

        # Forward
        out, _ = model(data)

        # Loss on Train Split Only
        loss = criterion(out[data.train_mask], data.y[data.train_mask])

        loss.backward()
        optimizer.step()

        # Validation every 50 epochs
        if epoch % 50 == 0:
            model.eval()
            pred = out.argmax(dim=1)

            # Use Test Mask for Validation Check
            y_true = data.y[data.test_mask].cpu().numpy()
            y_pred = pred[data.test_mask].cpu().numpy()

            # Metrics for Class 1 (Illicit)
            f1 = f1_score(y_true, y_pred, pos_label=1, zero_division=0)
            rec = recall_score(y_true, y_pred, pos_label=1, zero_division=0)

            print(f'Epoch {epoch:>3} | Loss: {loss:.4f} | Test F1: {f1:.4f} | Recall: {rec:.4f}')

    return model

In [ ]:
# --- CONFIGURATION ---
args = {
    'lr': 0.01,
    'weight_decay': 0.0005,
    'epochs': 1000,
    'hidden_units': 512,
    'num_classes': 2,
    'num_hops': [2, 2] # K=2
}

# 1. Load Data
data = load_data_production('.', apply_reduction=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)
print(f"   Using Device: {device}")

# 2. Initialize Model
model = ChebyshevConvolutionLin(
    args,
    kernel=args['num_hops'],
    num_features=data.num_features,
    hidden_units=args['hidden_units']
).to(device)

# 3. Calculate Weights (Illicit is Class 1)
# Weight 0.3 for Licit, 0.7 for Illicit (or 1.0 vs 3.0)
weights = torch.tensor([1.0, 3.0]).to(device)

# 4. Run Training
trained_model = train_production(args, model, data, class_weights=weights)

# 5. Save Model
torch.save(trained_model.state_dict(), 'cheb_model_production.pth')
print("✅ Model Saved! Download 'cheb_model_production.pth' from the files tab.")

🔄 Loading Elliptic Dataset...


/tmp/ipython-input-1136638605.py:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_classes['class'] = df_classes['class'].replace({'unknown': 3, '1': 1, '2': 0})


   Original Node Count: 203769
   Labeled Node Count:  46564
✅ Data Ready! Features: 165
   Using Device: cuda
🚀 Training for 1000 epochs...
Epoch   0 | Loss: 0.7641 | Test F1: 0.1371 | Recall: 0.6574
Epoch  50 | Loss: 0.1211 | Test F1: 0.5615 | Recall: 0.5439
Epoch 100 | Loss: 0.0585 | Test F1: 0.5684 | Recall: 0.4968
Epoch 150 | Loss: 0.0464 | Test F1: 0.5781 | Recall: 0.5485
Epoch 200 | Loss: 0.0398 | Test F1: 0.6419 | Recall: 0.5337
Epoch 250 | Loss: 0.0364 | Test F1: 0.5768 | Recall: 0.5633
Epoch 300 | Loss: 0.0479 | Test F1: 0.6908 | Recall: 0.6427
Epoch 350 | Loss: 0.0336 | Test F1: 0.6198 | Recall: 0.5540
Epoch 400 | Loss: 0.0300 | Test F1: 0.5942 | Recall: 0.5476
Epoch 450 | Loss: 0.0329 | Test F1: 0.6736 | Recall: 0.6537
Epoch 500 | Loss: 0.0804 | Test F1: 0.5021 | Recall: 0.5088
Epoch 550 | Loss: 0.0454 | Test F1: 0.5283 | Recall: 0.4700
Epoch 600 | Loss: 0.0368 | Test F1: 0.5968 | Recall: 0.6177
Epoch 650 | Loss: 0.0323 | Test F1: 0.6158 | Recall: 0.5919
Epoch 700 | Loss: 0